In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from pathlib import Path

In [6]:
this_dir = Path().resolve()
DATA_DIR = this_dir.parent / "data"
print(DATA_DIR)

/home/mikew/extracover/data


In [7]:
df = pd.read_csv(DATA_DIR / "ml_rows_last_two.csv")

In [8]:
X = df[
    [
        "innings",
        "ball_of_innings",
        # "in_powerplay",
        # "in_last_two",
        "wickets_down",
        "run_rate",
        "req_rate",
        "batter_in_first_10",
        "batter_strike_rate",
        "bowler_economy",
        "bowler_wicket_prob",
        "bowler_wide_noball_rate",
    ]
]
y = df[
    [
        "outcome",
        # "wide",
        # "noball",
        # "bye",
        # "legbye",
        # "wicket",
        # "dot_ball",
        # "single",
        # "two",
        # "three",
        # "four",
        # "six",
    ]
]

In [9]:
print(f"{df.shape=}")
print(f"{X.shape=}")
print(f"{y.shape=}")

df.shape=(177105, 11)
X.shape=(177105, 10)
y.shape=(177105, 1)


In [10]:
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)
print(f"{X_scaled.shape=}")

X_scaled.shape=(177105, 10)


In [11]:
from sklearn.cluster import KMeans

import joblib

for clusters in [50, 100, 200, 500, 1000]:
    clf = KMeans(n_clusters=clusters)
    clf.fit(X_scaled)
    joblib.dump(clf, DATA_DIR / f"Kmeans_fitted_{clusters}_clusters.model")